# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.5 which is incompatible.


The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726624e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [5]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.14849999668076636


In [6]:
%load_ext autoreload
%autoreload 2

In [7]:

from ddpg_agent import Agent
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import torch, pickle

In [8]:
def ddpg(n_episodes=500, max_t=1000, print_every=1):
    """Deep Deterministic Policy Gradient (DDPG)
    
    Params
    ======
        n_episodes (int)      : maximum number of training episodes
        max_t (int)           : maximum number of timesteps per episode
        train_mode (bool)     : if 'True' set environment to training mode
        solved_score (float)  : min avg score over consecutive episodes
        consec_episodes (int) : number of consecutive episodes used to calculate score
        print_every (int)     : interval to display results
        actor_path (str)      : directory to store actor network weights
        critic_path (str)     : directory to store critic network weights

    """
    
    scores_window = deque(maxlen=100)  # mean scores from most recent episodes
    scores_episodes = []                               # list of moving averages
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name] # reset environment
        states = env_info.vector_observations                   # get current state for each agent      
        scores = np.zeros(num_agents)                           # initialize score for each agent
        agent.reset()
        start_time = time.time()
        for t in range(max_t):
            
            print('\rEpisode {} TimeStep {}'.format(i_episode, t), end="")
            actions = agent.act(states, add_noise=True)         # select an action
            env_info = env.step(actions)[brain_name]            # send actions to environment
            next_states = env_info.vector_observations          # get next state
            rewards = env_info.rewards                          # get reward
            dones = env_info.local_done                         # see if episode has finished
            # save experience to replay buffer, perform learning step at defined interval
            for state, action, reward, next_state, done in zip(states, actions, rewards, next_states, dones):
                agent.step(state, action, reward, next_state, done, t)             
            states = next_states
            scores += rewards        
            if np.any(dones):                                   # exit loop when episode ends
                break

        diff = time.time() - start_time
        
        score_mean = np.mean(scores)
        scores_window.append(score_mean)         # save mean score to window
        scores_episodes.append(score_mean)    # save moving average
        
        window_mean = np.mean(scores_window)
        if i_episode % print_every == 0:
            print('\rEpisode {} ({} sec) \tScore: {:.1f}'.format(\
                  i_episode, round(diff), window_mean))
        
        torch.save(agent.actor_local.state_dict(), "actor.pth")
        torch.save(agent.critic_local.state_dict(), "critic.pth") 
        
        
        if np.mean(scores_window) >= 30:
            print('\nEnvironment SOLVED in {} episodes!\tScore ={:.1f}'.format(\
                                    i_episode, window_mean ))            
            
            torch.save(agent.actor_local.state_dict(), "actor.pth")
            torch.save(agent.critic_local.state_dict(), "critic.pth")  
            break
            
    return scores_window, scores_episodes

In [ ]:
agent = Agent(state_size=state_size, action_size=action_size, random_seed=1)
scores_window, scores_episodes = ddpg()

Episode 1 (115 sec) 	Score: 0.8
Episode 2 (118 sec) 	Score: 1.4
Episode 3 (119 sec) 	Score: 1.7
Episode 4 (119 sec) 	Score: 1.9
Episode 5 (119 sec) 	Score: 2.2
Episode 6 (121 sec) 	Score: 2.4
Episode 7 (121 sec) 	Score: 2.7
Episode 8 (122 sec) 	Score: 3.1
Episode 9 (122 sec) 	Score: 3.4
Episode 10 (123 sec) 	Score: 3.7
Episode 11 (125 sec) 	Score: 4.1
Episode 12 (126 sec) 	Score: 4.5
Episode 13 (128 sec) 	Score: 4.8
Episode 14 (130 sec) 	Score: 5.1
Episode 15 (132 sec) 	Score: 5.5
Episode 16 (134 sec) 	Score: 6.0
Episode 17 (135 sec) 	Score: 6.4
Episode 18 (138 sec) 	Score: 6.9
Episode 19 (139 sec) 	Score: 7.3
Episode 20 (142 sec) 	Score: 7.9
Episode 21 (144 sec) 	Score: 8.3
Episode 22 (147 sec) 	Score: 8.6
Episode 23 (149 sec) 	Score: 8.9
Episode 24 (151 sec) 	Score: 9.3
Episode 25 (153 sec) 	Score: 9.7
Episode 26 (156 sec) 	Score: 10.0
Episode 27 (160 sec) 	Score: 10.3
Episode 28 (162 sec) 	Score: 10.7
Episode 29 (164 sec) 	Score: 11.0
Episode 30 (167 sec) 	Score: 11.4
Episode 31 (17

In [9]:
agent = Agent(state_size=state_size, action_size=action_size, random_seed=1)

agent.actor_local.load_state_dict(torch.load('actor.pth'))
agent.critic_local.load_state_dict(torch.load('critic.pth'))

agent.actor_target.load_state_dict(torch.load('actor.pth'))
agent.critic_target.load_state_dict(torch.load('critic.pth'))

        
scores_window, scores_episodes = ddpg()

Episode 1 (116 sec) 	Score: 30.1

Environment SOLVED in 1 episodes!	Score =30.1


When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 